In [1]:
from pandas import DataFrame, Series
import pandas as pd; import numpy as np

In [2]:
#TODO: Update to useful data! This is just to get test the queries on userframe and revframe
import json
USERPATH='../full_dataset/user.json'
FS1PATH='../full_dataset/featureset1_resto_food_chinese.json'
userframe = DataFrame([json.loads(line) for line in open(USERPATH)])[['user_id', 'friends']]
active_users_frame = pd.read_json(FS1PATH)
userframe = pd.merge(userframe, active_users_frame, on='user_id')[['user_id', 'friends']]
active_users_frame = None

In [3]:
#Flatten "friends" field
flatteningframe = DataFrame([[i, friend]
                                     for i, friends in userframe['friends'].apply(list).iteritems()
                                         for friend in friends],
                                columns=['index', 'friend_id'])
flatteningframe = flatteningframe.set_index('index')
graph_base_frame = pd.merge(userframe, flatteningframe, left_index = True, right_index = True)[['user_id', 'friend_id']]

In [4]:
#Prune relationships
graph_base_frame = pd.merge(graph_base_frame, userframe, left_on='friend_id', right_on='user_id')
graph_base_frame = DataFrame(graph_base_frame.loc[graph_base_frame['user_id_y'] != None][['user_id_x', 'friend_id']])
graph_base_frame.columns = ['user_id', 'friend_id']

1149148

In [7]:
len(graph_base_frame.index)


287287